# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [2]:
import syft as sy
import torch as t
hook = sy.TorchHook(t)

In [3]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

In [4]:
x = t.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [5]:
x = x.share(bob, secure_worker)

In [8]:
bob._objects

{95965099713: tensor([1371826844651140449, 1421994283167056110, 2519995833896606010,
           51468845433392417,  145438730797377435])}

In [9]:
y = x + x

In [10]:
y.get()

tensor([ 2,  4,  6,  8, 10])

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [11]:
x = t.tensor([0.1,0.2,0.3,0.4,0.5])
x

tensor([0.1000, 0.2000, 0.3000, 0.4000, 0.5000])

In [12]:
x = x.fix_prec()

In [13]:
x

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300, 400, 500])

In [15]:
x = x.float_prec()
x

tensor([0.1000, 0.2000, 0.3000, 0.4000, 0.5000])

In [16]:
x =x.fix_prec()
x

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300, 400, 500])

In [17]:
type(x)

syft.frameworks.torch.tensors.interpreters.native.Tensor

In [18]:
type(x.child)

syft.frameworks.torch.tensors.interpreters.precision.FixedPrecisionTensor

In [19]:
type(x.child.child)#this is our data

syft.frameworks.torch.tensors.interpreters.native.Tensor

In [22]:
x.child.child

tensor([100, 200, 300, 400, 500])

In [23]:
y = x+x

In [24]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000, 0.8000, 1.0000])

### Shared Fixed Precision

And of course, we can combine the two!

In [25]:
x = t.tensor([0.1,0.2,0.3]).fix_prec().share(bob,alice,secure_worker)

In [26]:
x

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:54459433586 -> bob:13447325439]
	-> (Wrapper)>[PointerTensor | me:21967670313 -> alice:52929788876]
	-> (Wrapper)>[PointerTensor | me:66789097141 -> secure_worker:68728129924]
	*crypto provider: me*

In [31]:
y = x + x

In [32]:
y

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:348538757 -> bob:34946250793]
	-> (Wrapper)>[PointerTensor | me:86289015656 -> alice:76165406234]
	-> (Wrapper)>[PointerTensor | me:5925021057 -> secure_worker:72566955865]
	*crypto provider: me*

In [33]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])